In [1]:
import pandas as pd
import numpy as np
import sqlite3
import io
def adapt_array(arr):
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

def convert_array(text):
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)

sqlite3.register_adapter(np.ndarray, adapt_array)
sqlite3.register_converter("array", convert_array)


conn = sqlite3.connect("chemdatabase_copy.db", detect_types=sqlite3.PARSE_DECLTYPES)
cursor = conn.cursor()


In [13]:
import yaml

with open("config_endpoints.yml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile)


In [19]:
endpoints_list = cfg["endpoints_list"]

In [20]:
endpoints_list

['o_mus_ipr_LD',
 'o_rat_orl_TDLo',
 'o_mus_ipr_LDLo',
 'o_mus_orl_TDLo',
 'o_rat_ipr_TDLo']

In [ ]:
select * from endpoints 

In [ ]:
cursor.execute("select * from experimental_dat where ")

In [16]:
from create_database import open_database

In [17]:
open_database(cfg["main"]["db_name"])

(<sqlite3.Connection at 0x7f694a0d92d0>, <sqlite3.Cursor at 0x7f694a1e1960>)

In [23]:
cfg['endpoints_list']

['o_mus_ipr_LD',
 'o_rat_orl_TDLo',
 'o_mus_ipr_LDLo',
 'o_mus_orl_TDLo',
 'o_rat_ipr_TDLo']

In [3]:

for section in cfg:
    print(section)
print(cfg['mysql'])
print(cfg['other'])

mysql
other
{'host': 'localhost', 'user': 'root', 'passwd': 'my secret password', 'db': 'write-math'}
{'preprocessing_queue': ['preprocessing.scale_and_center', 'preprocessing.dot_reduction', 'preprocessing.connect_lines'], 'use_anonymous': True}


In [4]:
cfg['mysql']

{'host': 'localhost',
 'user': 'root',
 'passwd': 'my secret password',
 'db': 'write-math'}

In [7]:
cfg['preprocessing_queue']

KeyError: 'preprocessing_queue'

In [8]:
cfg

{'mysql': {'host': 'localhost',
  'user': 'root',
  'passwd': 'my secret password',
  'db': 'write-math'},
 'other': {'preprocessing_queue': ['preprocessing.scale_and_center',
   'preprocessing.dot_reduction',
   'preprocessing.connect_lines'],
  'use_anonymous': True}}

In [30]:
temp =cursor.execute("select * from endpoints").fetchall()
temp = [(data[0], data[1].strip()+"_"+data[2].strip()) for data in temp]
NameOfEndpoint2db_index = {data[1]:data[0] for data in temp}


In [32]:
NameOfEndpoint2db_index['o_mus_ipr_LD']

1

In [ ]:
['o_mus_ipr_LD']

In [33]:
indexes = [NameOfEndpoint2db_index[endpoint] for endpoint in cfg['endpoints_list']]

In [34]:
indexes

[1, 2, 3, 4, 5]

In [39]:
"("+", ".join(str(x) for x in indexes )+")"

'(1, 2, 3, 4, 5)'

In [114]:
temp =cursor.execute("select * from experimental_data where id_endpoint in "+("("+", ".join(str(x) for x in indexes )+")")).fetchall()

In [184]:
temp[:5]

[(992, 1, 4.41),
 (10518, 1, 3.99),
 (61767, 1, 3.8),
 (44985, 1, 3.8),
 (34192, 1, 3.49)]

In [116]:
endpoints_dict = {}

In [117]:
for data in temp:
    if data[0] in endpoints_dict.keys():
        endpoints_dict[data[0]].append((data[1],data[2]))
    else:
        endpoints_dict[data[0]] = [(data[1],data[2])]

In [118]:
indexe = [x+1  for x in list(range(len(endpoints_list)))]

In [122]:
for key in endpoints_dict.keys():
    vals = {idx: None for idx in indexe}
    for elem in endpoints_dict[key]:
        vals[elem[0]] = elem[1]
    endpoints_dict[key] = list(vals.values())

In [123]:
db_smiles = cursor.execute("select * from molecules").fetchall()

In [124]:
index2smiles = {smile[0]: smile[3] for smile in db_smiles}

In [125]:
endpoints_keys = list(endpoints_dict.keys())
for key in endpoints_keys:
    endpoints_dict[index2smiles[key]] = endpoints_dict.pop(key)

In [127]:
endpoints_list

['o_mus_ipr_LD',
 'o_rat_orl_TDLo',
 'o_mus_ipr_LDLo',
 'o_mus_orl_TDLo',
 'o_rat_ipr_TDLo']

In [159]:
df = pd.DataFrame.from_dict(endpoints_dict,orient="index", columns=endpoints_list)

In [134]:
df_tox = pd.read_csv("toxicity_85832.csv")

In [150]:
df_tox = df_tox.set_index("SMILES", )

In [148]:
df_tox = df_tox.reset_index()

In [183]:
df_tox.head()

,o_mus_ipr_LD,o_rat_orl_TDLo,o_mus_ipr_LDLo,o_mus_orl_TDLo,o_rat_ipr_TDLo,o_mus_ivn_LD,o_rat_ipr_LD,o_mus_orl_LD,o_mus_unr_LD,o_rat_unr_LD,...,o_rbt_orl_LD,o_rbt_ivn_LD,o_rat_ivn_LD,o_mus_orl_LDLo,o_rat_skn_LD,o_mam_unr_LD,o_gpg_orl_LD,o_wmn_orl_TDLo,o_man_orl_TDLo,o_rat_scu_TDLo
SMILES,,,,,,,,,,,,,,,,,,,,,
Br.Br.Br.Br.N=C(N)SCCN(CCSC(=N)N)CCSC(=N)N,4.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Br.Br.Br.CCN(CCSC(=N)N)CCSC(=N)N,3.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Br.Br.Br.CN(CCSC(=N)N)CCSC(=N)N,3.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Br.Br.Br.CNc1cnc(C)c(N)c1CSC(=N)N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.04,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Br.Br.Br.COc1ccc2[nH]cc(CCNC(=O)C(N)CSSCC(N)C(=O)NCCc3c[nH]c4ccc(OC)cc34)c2c1,NaN,NaN,NaN,NaN,NaN,4.27,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [157]:
df_tox.loc["Br.Br.Br.Br.N=C(N)SCCN(CCSC(=N)N)CCSC(=N)N"]

o_mus_ipr_LD       4.41
o_rat_orl_TDLo      NaN
o_mus_ipr_LDLo      NaN
o_mus_orl_TDLo      NaN
o_rat_ipr_TDLo      NaN
o_mus_ivn_LD        NaN
o_rat_ipr_LD        NaN
o_mus_orl_LD        NaN
o_mus_unr_LD        NaN
o_rat_unr_LD        NaN
o_mus_scu_LDLo      NaN
o_rat_scu_LD        NaN
o_mus_scu_LD        NaN
o_rat_ipr_LDLo      NaN
o_mus_ipr_TDLo      NaN
o_rbt_skn_LD        NaN
o_rat_orl_LD        NaN
o_rat_ivn_TDLo      NaN
o_rat_orl_LDLo      NaN
o_rbt_orl_LD        NaN
o_rbt_ivn_LD        NaN
o_rat_ivn_LD        NaN
o_mus_orl_LDLo      NaN
o_rat_skn_LD        NaN
o_mam_unr_LD        NaN
o_gpg_orl_LD        NaN
o_wmn_orl_TDLo      NaN
o_man_orl_TDLo      NaN
o_rat_scu_TDLo      NaN
Name: Br.Br.Br.Br.N=C(N)SCCN(CCSC(=N)N)CCSC(=N)N, dtype: float64

In [161]:
df.loc["Br.Br.Br.Br.N=C(N)SCCN(CCSC(=N)N)CCSC(=N)N"]

o_mus_ipr_LD      4.41
o_rat_orl_TDLo     NaN
o_mus_ipr_LDLo     NaN
o_mus_orl_TDLo     NaN
o_rat_ipr_TDLo     NaN
Name: Br.Br.Br.Br.N=C(N)SCCN(CCSC(=N)N)CCSC(=N)N, dtype: float64

In [163]:
import numpy.ma as ma

In [164]:
class OurRobustToNanScaler():
    """
    This class is equal to StandardScaler from sklearn but can work with NaN's (ignoring it) but
    sklearn's scaler can't do it.
    """
    def fit(self, data):
        masked = ma.masked_invalid(data)
        self.means = np.mean(masked, axis=0)
        self.stds = np.std(masked, axis=0)
    
    def fit_transform(self, data):
        self.fit(data)
        masked = ma.masked_invalid(data)
        masked -= self.means
        masked /= self.stds
        return ma.getdata(masked)

    def inverse_transform(self, data):
        masked = ma.masked_invalid(data)
        masked *= self.stds
        masked += self.means
        return ma.getdata(masked)

In [165]:
df.values

array([[4.41,  nan,  nan,  nan,  nan],
       [3.99,  nan,  nan,  nan,  nan],
       [3.8 ,  nan,  nan,  nan,  nan],
       ...,
       [2.83,  nan,  nan,  nan,  nan],
       [2.26,  nan,  nan,  nan,  nan],
       [2.7 ,  nan,  nan,  nan,  nan]])

In [166]:
scaler = OurRobustToNanScaler()

In [168]:
scaler.fit(df.values)

In [170]:
new_vals = scaler.fit_transform(df.values)

In [175]:
new_vals

array([[ 1.96798376,         nan,         nan,         nan,         nan],
       [ 1.36189809,         nan,         nan,         nan,         nan],
       [ 1.08771648,         nan,         nan,         nan,         nan],
       ...,
       [-0.31205281,         nan,         nan,         nan,         nan],
       [-1.13459764,         nan,         nan,         nan,         nan],
       [-0.49965075,         nan,         nan,         nan,         nan]])

In [176]:
df.values = new_vals


/home/workstation/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


AttributeError: can't set attribute

In [181]:
new_vals

array([[ 1.96798376,         nan,         nan,         nan,         nan],
       [ 1.36189809,         nan,         nan,         nan,         nan],
       [ 1.08771648,         nan,         nan,         nan,         nan],
       ...,
       [-0.31205281,         nan,         nan,         nan,         nan],
       [-1.13459764,         nan,         nan,         nan,         nan],
       [-0.49965075,         nan,         nan,         nan,         nan]])

In [179]:
df[:][:] = new_vals

In [182]:
df.head()

,o_mus_ipr_LD,o_rat_orl_TDLo,o_mus_ipr_LDLo,o_mus_orl_TDLo,o_rat_ipr_TDLo
Br.Br.Br.Br.N=C(N)SCCN(CCSC(=N)N)CCSC(=N)N,1.967984,NaN,NaN,NaN,NaN
Br.Br.Br.CCN(CCSC(=N)N)CCSC(=N)N,1.361898,NaN,NaN,NaN,NaN
Br.Br.Br.CN(CCSC(=N)N)CCSC(=N)N,1.087716,NaN,NaN,NaN,NaN
Br.Br.Br.N=C(N)SCCCNCCNCC12CC3CC(CC(C3)C1)C2,1.087716,NaN,NaN,NaN,NaN
Br.Br.Br.N=C(N)SCCNCCSC(=N)N,0.640368,NaN,NaN,NaN,NaN


In [185]:
smiles_ind = list(set([data[0] for data in temp]))

In [201]:

descriptors = cursor.execute("select * from descriptors_values where id_molecule in "+("("+", ".join(str(x) for x in smiles_ind )+")")).fetchall()

In [210]:
ind2descriptors = {index2smiles[data[0]]:np.nan_to_num(data[4]) for data in descriptors}

In [227]:
smiles2descriptors = {index2smiles[data[0]]:np.nan_to_num(data[4])[0].tolist() for data in descriptors}

In [213]:
df.head()

,o_mus_ipr_LD,o_rat_orl_TDLo,o_mus_ipr_LDLo,o_mus_orl_TDLo,o_rat_ipr_TDLo
Br.Br.Br.Br.N=C(N)SCCN(CCSC(=N)N)CCSC(=N)N,1.967984,NaN,NaN,NaN,NaN
Br.Br.Br.CCN(CCSC(=N)N)CCSC(=N)N,1.361898,NaN,NaN,NaN,NaN
Br.Br.Br.CN(CCSC(=N)N)CCSC(=N)N,1.087716,NaN,NaN,NaN,NaN
Br.Br.Br.N=C(N)SCCCNCCNCC12CC3CC(CC(C3)C1)C2,1.087716,NaN,NaN,NaN,NaN
Br.Br.Br.N=C(N)SCCNCCSC(=N)N,0.640368,NaN,NaN,NaN,NaN


In [228]:
df_descriptors = pd.DataFrame.from_dict(smiles2descriptors, orient="index")

KeyboardInterrupt: 

In [226]:
smiles2descriptors["Br.Br.Br.Br.N=C(N)SCCN(CCSC(=N)N)CCSC(=N)N"].tolist()

[13.384261131286621,
 12.092146873474121,
 0.0,
 7.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 48.0,
 23.0,
 0.0,
 0.0,
 14.0,
 25.0,
 0.0,
 9.0,
 7.0,
 0.0,
 3.0,
 0.0,
 0.0,
 0.0,
 4.0,
 0.0,
 4.0,
 173.53086853027344,
 138.0,
 120.0,
 100.0,
 122.0,
 161.0,
 133.3333282470703,
 100.0,
 76.0,
 103.0,
 111.0,
 165.0,
 183.0,
 189.0,
 198.0,
 189.0,
 156.0,
 147.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 6360.0,
 1337.0,
 1722.0,
 1377.0,
 1398.0,
 1530.0,
 1845.0,
 1428.0,
 1764.0,
 31319.279296875,
 4851.900390625,
 6350.498046875,
 4581.65234375,
 4917.26513671875,
 5207.34716796875,
 6658.60888671875,
 5031.36279296875,
 6414.9287109375,
 10895.91015625,
 9927.072265625,
 12056.8408203125,
 12666.62109375,
 13253.900390625,
 13410.376953125,
 12909.98828125,
 12397.837890625,
 11114.6953125,
 374.9170837402344,
 343.5733947753906,
 488.7846984863281,
 596.7127685546875,
 648.143798828125,
 778.138916015625,
 630.17297363281

In [225]:
smile

NameError: name 'smil' is not defined